In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class QLearningAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.99):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma
        self.q_values = torch.zeros((state_size, action_size), dtype=torch.float32, requires_grad=True)
        self.optimizer = optim.Adam([self.q_values], lr=learning_rate)

    def select_action(self, state):
        with torch.no_grad():
            return torch.argmax(self.q_values[state]).unsqueeze(0)

    def update_q_values(self, state, action, reward, next_state):
        q_value = self.q_values[state, action]
        next_max_q_value = torch.max(self.q_values[next_state]).item()
        target_q_value = reward + self.gamma * next_max_q_value
        loss = nn.MSELoss()(q_value, target_q_value)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

def add_noise(images, mean=0, std=0.1):
    noise = torch.randn(images.size()) * std + mean
    return images + noise

def train_cnn(cnn, train_loader, criterion, optimizer, epoch, print_freq=100):
    cnn.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader, 1):
        optimizer.zero_grad()
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        if i % print_freq == 0:
            print(f"Epoch {epoch}, Example {i}, Loss: {running_loss / print_freq:.4f}, Acc: {100 * correct / total:.2f}%")
            running_loss = 0.0

def evaluate_cnn(cnn, dataloader):
    cnn.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = cnn(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy : {accuracy:.2f}%")

transform = transforms.Compose([transforms.ToTensor()])
cifar_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(cifar_dataset))
train_dataset, val_dataset = random_split(cifar_dataset, [train_size, len(cifar_dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

cnn = CNN()
q_agent = QLearningAgent(state_size=10, action_size=10)
criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.Adam(cnn.parameters(), lr=0.001)

for epoch in range(5):
    train_cnn(cnn, train_loader, criterion, cnn_optimizer, epoch)

    for i, (inputs, labels) in enumerate(train_loader, 1):
        cnn.eval()
        with torch.no_grad():
            cnn_predictions = cnn(inputs)
        state = torch.argmax(cnn_predictions, dim=1)
        action = q_agent.select_action(state)
        noisy_inputs = add_noise(inputs)
        q_agent_predictions = cnn(noisy_inputs)
        reward = torch.eq(torch.tensor(action), torch.argmax(q_agent_predictions, dim=1)).float()
        next_state = torch.argmax(q_agent_predictions, dim=1)
        q_agent.update_q_values(state, action, reward, next_state)
        cnn.train()
        cnn_optimizer.zero_grad()
        cnn_outputs = cnn(noisy_inputs)
        cnn_loss = criterion(cnn_outputs, torch.argmax(q_agent_predictions, dim=1))
        cnn_loss.backward()
        cnn_optimizer.step()

    evaluate_cnn(cnn, val_loader)
